# Tests for quering local PBF osm files

guide for using the filter https://pypi.org/project/esy-osmfilter/


## imports

In [ ]:
import configparser, contextlib
import os, sys

from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation

import urllib.request


## Get file

In [ ]:
c = 'liechtenstein-latest'
city_name = 'berlin-latest'

PBF_inputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.osm.pbf')
JSON_outputfile = os.path.join(os.getcwd(), f'test-osmfilter/{city_name}.json')

PBF_inputfile

In [ ]:
if not os.path.exists(f'test-osmfilter/{city_name}.osm.pbf'):
    filename, headers = urllib.request.urlretrieve(
        f'https://download.geofabrik.de/europe/{city_name}.osm.pbf',
        filename=f'test-osmfilter/{city_name}.osm.pbf')
# PBF_inputfile = filename

## Filter main data

In [ ]:
prefilter   = {Node: {}, Way: {"man_made":["pipeline",],}, Relation: {}}
blackfilter = [("pipeline","substation"),]
whitefilter = [(('man_made','pipeline'),('waterway','drain')),] #[(("waterway","drain"),("name","Wäschgräble")), ]

In [ ]:
>>> [Data,_]=run_filter('noname',
...                     PBF_inputfile, 
...                     JSON_outputfile, 
...                     prefilter,
...                     whitefilter, 
...                     blackfilter, 
...                     NewPreFilterData=True, 
...                     CreateElements=False, 
...                     LoadElements=False,
...                     verbose=True)

In [ ]:
>>> len(Data['Node'])

In [ ]:
>>> len(Data['Relation'])

In [ ]:
>>> len(Data['Way'])

In [ ]:
Data['Way']

## Filter out (blackfilter)

In [ ]:
>>> [_,Elements]=run_filter('funny-waterway-pipelines',
...                            PBF_inputfile, 
...                            JSON_outputfile, 
...                            prefilter,
...                            whitefilter, 
...                            blackfilter, 
...                            NewPreFilterData=False, 
...                            CreateElements=True, 
...                            LoadElements=False,
...                            verbose=True)
>>> len(Elements['funny-waterway-pipelines'])
0
>>> len(Elements['funny-waterway-pipelines']['Relation'])
0
>>> len(Elements['funny-waterway-pipelines']['Way'])

In [ ]:
Elements

## save geojson

In [ ]:
>>> from esy.osmfilter import export_geojson

# jsontype='Line' 'Point' 

>>> export_geojson(Elements['funny-waterway-pipelines']['Way'],Data,
... filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## open geosjon

In [ ]:
import geopandas as gpd

elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][0]

In [ ]:
elements.describe()

# shapely, create geometry from geojson

In [ ]:
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

# test

In [ ]:
green = {'landuse':['grass','forest','orchard','allotments','cementery','flowerbed', 'meadow','greenfield', 'recreation_ground','village_green'],
         'leisure':['park','playground','garden'],
         'natural':['heath','shrubbery','wood','grassland'],}
amenit = {'amenity':['bbq'],}

cycle = {'highway':['pedestrian','cycleway']}


prefilter   = {Node: {}, Way: cycle, Relation: {}}


[Data,_]=run_filter('noname',
                    PBF_inputfile, 
                    JSON_outputfile, 
                    prefilter,
                    whitefilter, 
                    blackfilter, 
                    NewPreFilterData=True, 
                    CreateElements=False, 
                    LoadElements=False,
                    verbose=True)

In [ ]:
ways = len(Data['Way'])
rels = len(Data['Relation'])
nodes = len(Data['Node'])
           
print(f"{ways} Ways")
print(f"{rels} Rels")
print(f"{nodes} Nodes")

In [ ]:
Data['Way']

In [ ]:
# jsontype='Line' 'Point' 

export_geojson(Data['Way'],Data,
filename=f'test-osmfilter/{city_name}.geojson',jsontype='Line')

## Create Shapely geomtry

In [ ]:
elements = gpd.read_file(f'test-osmfilter/{city_name}.geojson')
elements['geometry'][3]

In [ ]:
polygonize(elements['geometry'][1])

In [ ]:
lis = []
for l in elements['geometry']:
    lis.append(l)

In [ ]:
lis[2]

In [ ]:
type(lis[2])

In [ ]:
# converting to multi points

lis_p = geometry.MultiPoint(lis)


In [ ]:
# converting to multi lines

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
lines = geometry.MultiLineString(borders)

In [ ]:
lines

In [ ]:
# converting to mulpolygon

merged = linemerge([*lis])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
polygons = list(polygonize(borders))
parks = geometry.MultiPolygon(polygons)

In [ ]:
parks.geoms[3]

In [ ]:
type(parks)

## Create Geojson

In [ ]:
import folium
from shapely.geometry import mapping


In [ ]:
mapping(lines)

## example map in folium

In [ ]:
map_test = folium.Map(width=750,location=[52.5205,13.3988],height=500,tiles="openstreetmap",   zoom_start=11)
folium.GeoJson(mapping(lines)).add_to(map_test)

In [ ]:
map_test

# Final

## Imports

In [25]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import configparser, contextlib
import os, sys

# from esy.osmfilter import osm_colors as CC
from esy.osmfilter import run_filter 
from esy.osmfilter import Node, Way, Relation
from esy.osmfilter import export_geojson


import urllib.request

import geopandas as gpd

import folium

import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

from shapely.geometry import mapping
from geojson import dump

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from livablestreets.query_names_detailed import master_query
from livablestreets.query_names_detailed import master_query_complex

from livablestreets.query_osmfilter import data_from_pbf

In [27]:
df = master_query_complex()
df['geomtype'].iloc[0]

'Node'

In [28]:
df

,query_string,name,distance,geomtype,jsontype,shapelytype,category
public_transport_bus,"{'amenity': ['bus_station'], 'bus_bay': '', 'h...",bus,200,Node,Point,Point,mobility
public_transport_rail,"{'public_transport': ['stop_position', 'statio...",rail,500,Node,Point,Point,mobility
bike_infraestructure,"{'amenity': ['bicycle_parking', 'bicycle_repai...",bike,50,Node,Point,Point,mobility
eating,"{'amenity': ['cafe', 'restaurant', 'food_court...",eating,200,Node,Point,Point,social_life
night_life,"{'amenity': ['bar', 'pub', 'biergarten', 'nigh...",nightlife,300,Node,Point,Point,social_life
culture,"{'amenity': ['social_centre', 'theatre', 'publ...",culture,500,Node,Point,Point,social_life
community,"{'office': ['association', 'charity', 'coworki...",community,400,Node,Point,Point,social_life
health_regional,"{'amenity': ['hospital', 'clinic', 'nursing_ho...",health_regional,1000,Node,Point,Point,activities
health_local,"{'amenity': ['baby_hatch', 'dentist', 'doctors...",health_local,300,Node,Point,Point,activities
goverment,"{'amenity': ['townhall', 'courthouse']}",goverment,500,Node,Point,Point,activities


In [29]:
new = df.iloc[21:22]

In [30]:
new

,query_string,name,distance,geomtype,jsontype,shapelytype,category
water,"{'natural': ['water', 'beach'], 'amenity': ['f...",water,500,Way,Line,multiPolygon,comfort


In [31]:
df.index[0]

'public_transport_bus'

In [32]:
df['query_string'].iloc[0]

{'amenity': ['bus_station'], 'bus_bay': '', 'highway': ['bus_stop']}

## get file

https://download.geofabrik.de/europe/germany/berlin-latest.osm.pbf

In [33]:
def get_pbf_json(country,city):
    
    city_name = f'{city}-latest'
    
    if not os.path.exists(f'localfilter/{city_name}.osm.pbf'):
        filename, headers = urllib.request.urlretrieve(
            f'https://download.geofabrik.de/europe/{country}/{city_name}.osm.pbf',
            filename=f'localfilter/{city_name}.osm.pbf')
        PBF_inputfile = filename

    PBF_inputfile = os.path.join(os.getcwd(), f'localfilter/{city_name}.osm.pbf')
    JSON_outputfile = os.path.join(os.getcwd(), f'localfilter/{city_name}.json')

        
    return PBF_inputfile, JSON_outputfile

In [34]:
PBF_inputfile, JSON_outputfile = get_pbf_json('germany','berlin')

In [35]:
PBF_inputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.osm.pbf'

In [36]:
JSON_outputfile

'/home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.json'

## execute filter

In [37]:
def shapely_format(lis,shapelytype):
    if shapelytype == 'Point':
        # converting to multi points
        shapely_geom = geometry.MultiPoint(lis)

    if shapelytype == 'lineString':
    # converting to multi lines
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        shapely_geom = geometry.MultiLineString(lis)      

    if shapelytype == 'multiPolygon':
    #converting to multipolygon
        merged = linemerge([*lis])  # merge LineStrings
        borders = unary_union(merged)  # linestrings to a MultiLineString
        polygons = list(polygonize(borders))
        shapely_geom = geometry.MultiPolygon(lis)
        
    return shapely_geom


        
def run_filter(query_df,city_name):
    for index, row in query_df.iterrows():
        filter_name = index
        string = row['query_string']
        geometry = row['geomtype']
        
        jsontype = row['jsontype']
        category = row['category']
        shapelytype = row['shapelytype']
        
        if not os.path.exists(f'localfilter/{city_name}-{category}-{filter_name}.geojson'):
            #initilize filter
            print(f'Filtering fetures: {filter_name} - {category}')
            Data = data_from_pbf(filter_name, string, PBF_inputfile, JSON_outputfile)
            export_geojson(Data[geometry],Data,filename=f'localfilter/{city_name}-{category}-{filter_name}.geojson',jsontype=jsontype)
        
        if not os.path.exists(f'localfilter/shapes-{city_name}-{category}-{filter_name}.geojson'):
            print(f'Exporting shapes of: {filter_name} - {category} as: {geometry}')
            elements = gpd.read_file(f'localfilter/{city_name}-{category}-{filter_name}.geojson')
                    
            # create list of features
            lis = []
            for l in elements['geometry']:
                lis.append(l)

            shapely_geo = shapely_format(lis,shapelytype)

            gjson = mapping(shapely_geo)
            with open(f'localfilter/shapes-{city_name}-{category}-{filter_name}.geojson', 'w') as f:
                dump(gjson, f)




In [38]:
run_filter(new,'berlin')

INFO:esy.osmfilter.pre_filter:PreFilter OSM GAS DATA
INFO:esy.osmfilter.pre_filter:InputFile     : /home/laia/code/Laiagdla/livablestreets/notebooks/localfilter/berlin-latest.osm.pbf
INFO:esy.osmfilter.pre_filter:Size          : 70008           kbyte
INFO:esy.osmfilter.pre_filter:Estimated Time: 10.00           s
INFO:esy.osmfilter.pre_filter:=============================


Filtering fetures: water - comfort


INFO:esy.osmfilter.pre_filter:0.5
INFO:esy.osmfilter.pre_filter:1
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilt

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

...

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfilter.pre_filter:1.2
INFO:esy.osmfi

Exporting shapes of: water - comfort as: Way


TypeError: 'LineString' object is not subscriptable

In [ ]:
Data

# Cheking the data

In [28]:
import folium
from shapely.geometry import mapping
from folium.plugins import MarkerCluster


i = 20
city_name = 'berlin'
filter_name = df.index[i]
category = df['category'].iloc[i]

elements = gpd.read_file(f'localfilter/{city_name}-{category}-{filter_name}.geojson')

In [ ]:

map_test = folium.Map(width=750, height=400, location=[52.5205,13.3988], tiles="openstreetmap", zoom_start=14)
marker_cluster = MarkerCluster().add_to(map_test)

folium.GeoJson(mapping(elements)).add_to(marker_cluster)

loc = f'{city_name} {category} {filter_name}'
title_html = '''
             <h3 align="left" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
map_test.get_root().html.add_child(folium.Element(title_html))


map_test

In [19]:
elements = gpd.read_file(f'localfilter/{city_name}-{category}-{filter_name}.geojson')

In [22]:
elements.value_counts

<bound method DataFrame.value_counts of       FIXME access addr:city addr:country addr:housenumber addr:inclusion  \
0      None   None      None         None             None           None   
1      None   None      None         None             None           None   
2      None   None      None         None             None           None   
3      None   None      None         None             None           None   
4      None   None      None         None             None           None   
...     ...    ...       ...          ...              ...            ...   
18582  None   None      None         None             None           None   
18583  None   None      None         None             None           None   
18584  None   None      None         None             None           None   
18585  None   None      None         None             None           None   
18586  None   None      None         None             None           None   

      addr:postcode addr:street add